In [3]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import pandas as pd
import numpy
import cv2

In [4]:
model_path = 'C:/Projects/ISL-to-text/INCLUDE 50/MP_Models/pose_landmarker_heavy.task'
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
options = PoseLandmarkerOptions(base_options = BaseOptions(model_asset_path = model_path),running_mode=VisionRunningMode.IMAGE)

In [5]:
def vid2frames(p):
    frames = []
    vid = cv2.VideoCapture(p)
    suc, img = vid.read()
    count = 0
    while suc:
        frames.append(img)
        suc, img = vid.read()
        count += 1
    return frames, count
    

In [23]:
dir = "C:/Projects/ISL Research/INCLUDE50"
out_path = "C:/Projects/ISL Research"
# Name of dir where includ, name of path for output
types = os.listdir(dir)
for t in types:
    for labels in os.listdir(os.path.join(dir,t)):
        for l in labels:
            label = l[2:]
            for file in os.listdir(os.path.join(dir,t,label)):
                # os.mkdir(os.path.join(out_path,label,file+"frames"))
                frames = []
                vid = cv2.VideoCapture(os.path.join(dir,t,label,file))
                suc, img = vid.read()
                count = 1
                while suc:
                    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)
                    with PoseLandmarker.create_from_options(options) as landmarker:
                        pose_landmarker_result = landmarker.detect(mp_image)
                    print(pose_landmarker_result.x)
                    print("hello")
                    cv2.imshow('image', img)
                    cv2.imwrite(os.path.join(out_path,label,file+"frames",str(count)+".jpeg"),img)
                    frames.append(img)
                    suc, img = vid.read()
                    count += 1
                    break
    break

                

This gives me all the frames of all the videos in the label. and stores these images in a folder